In [ ]:
import geopandas as gpd

In [ ]:
gdf=gpd.read_file('data/grilla_mdt_urbano_epsg4326.geojson')

In [ ]:
gdf.plot()

In [ ]:
gdf

In [ ]:
gdf.info()

# Time Lapse

In [ ]:
import ee
import geemap

In [ ]:
geemap.ee_initialize()

In [ ]:
Map = geemap.Map(center=[42.435677739012725, 129.74609161652995], zoom=12)
Map

In [ ]:
roi = Map.user_roi
if roi is None:
    roi = ee.Geometry.BBox(-74.7222, -8.5867, -74.1596, -8.2824)
    Map.addLayer(roi)
    Map.centerObject(roi)

In [ ]:
timelapse = geemap.landsat_timelapse(
    roi,
    out_gif='north_korea_huirong.gif',
    start_year=1984,
    end_year=2022,
    start_date='01-01',
    end_date='12-31',
    bands=['SWIR1', 'NIR', 'Red'],
    frames_per_second=5,
    title='Landsat Timelapse',
    progress_bar_color='blue',
    mp4=True,
)
geemap.show_image(timelapse)

# Landcover

In [15]:
import ee
import geemap

In [16]:
geemap.ee_initialize()

In [17]:
Map = geemap.Map(center=[42.442501697577775, 129.7403904429394], zoom=12)

dataset = ee.ImageCollection("ESA/WorldCover/v100").first()
Map.addLayer(dataset, {'bands': ['Map']}, "Landcover")

ee_class_table = """
Value	Color	Description
10	006400	Trees
20	ffbb22	Shrubland
30	ffff4c	Grassland
40	f096ff	Cropland
50	fa0000	Built-up
60	b4b4b4	Barren / sparse vegetation
70	f0f0f0	Snow and ice
80	0064c8	Open water
90	0096a0	Herbaceous wetland
95	00cf75	Mangroves
100	fae6a0	Moss and lichen
"""

legend_dict = geemap.legend_from_ee(ee_class_table)
Map.add_legend(title="ESA Land Cover", legend_dict=legend_dict)
Map

Map(center=[42.442501697577775, 129.7403904429394], controls=(WidgetControl(options=['position', 'transparent_…

# Split map pre and post

In [1]:
import ee
import geemap
geemap.ee_initialize()

In [19]:

# Define the center of your map
center = [42.44069022935791, 129.73017697955964]

# Create a Map
Map = geemap.Map(center=center, zoom=12)
Map


Map(center=[42.44069022935791, 129.73017697955964], controls=(WidgetControl(options=['position', 'transparent_…

In [21]:
# Get the bounding box (roi) from the map
roi = Map.user_roi

# Optionally, you can add the ROI back to the map to visualize it:
Map.addLayer(roi, {}, 'ROI')
Map


Map(bottom=775331.0, center=[42.44081432448306, 129.70587730407718], controls=(WidgetControl(options=['positio…

In [28]:
# Define a bounding box around the center
# roi = ee.Geometry.Point(center).buffer(1000).bounds()

# Fetch pre and post flood images
pre_flood_collection = (
    ee.ImageCollection('COPERNICUS/S2')
    .filterBounds(roi)
    .filterDate(ee.Date('2012-01-01'), ee.Date('2016-01-01'))
    .sort('CLOUDY_PIXEL_PERCENTAGE')
)

post_flood_collection = (
    ee.ImageCollection('COPERNICUS/S2')
    .filterBounds(roi)
    .filterDate(ee.Date('2016-02-01'), ee.Date('2020-12-31'))
    .sort('CLOUDY_PIXEL_PERCENTAGE')
)

# Take the first (least cloudy) image from the collections
pre_flood_image = pre_flood_collection.first()
post_flood_image = post_flood_collection.first()


In [29]:
# Set visualization parameters
vis_params = {
    'bands': ['B4', 'B3', 'B2'], 
    'min': 1, 
    'max': 3000 
    # 'gamma': 1.4
}

# Create a split panel map
left_layer = geemap.ee_tile_layer(pre_flood_image, vis_params, 'Pre Flood')
right_layer = geemap.ee_tile_layer(post_flood_image, vis_params, 'Post Flood')

Map.split_map(left_layer, right_layer)
Map.addLayer(roi)  # Optionally, add the ROI as a layer on the map
Map


EEException: Image.visualize: Parameter 'image' is required.

In [30]:
# Check the size of the collections
print('Pre-flood collection size:', pre_flood_collection.size().getInfo())
print('Post-flood collection size:', post_flood_collection.size().getInfo())


Pre-flood collection size: 0
Post-flood collection size: 597
